In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import cv2
import typing
import numpy as np
import pandas as pd
from tqdm import tqdm
from drive.MyDrive.mltu.configs import BaseModelConfigs

from drive.MyDrive.mltu.inferenceModel import OnnxInferenceModel
from drive.MyDrive.mltu.utils.text_utils import ctc_decoder, get_cer

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, char_list: typing.Union[str, list], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.char_list = char_list

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.char_list)[0]

        return text

def predict(X):
  y = []
  for i in range(len(X_test)):
    image = cv2.imread('drive/MyDrive/dataset/test/' + X[i])
    prediction = model.predict(image)
    y.append(prediction)
  return np.array(y)

if __name__ == "__main__":

    configs = BaseModelConfigs.load("drive/MyDrive/Captcha_models/0816101_model/configs.yaml")
    model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

    df_test = pd.DataFrame(pd.read_csv("drive/MyDrive/my_final/training/dataset/test/sample_submission.csv"))
    X_test = df_test["filename"].to_numpy()

    df_test["label"] = predict(X_test)
    df_test.to_csv("0816101_prediction.csv")